# Tutorial 01 - Calculating Magnitudes of Evoked Compound Action Potentials

We have included several methods to allow the user to calculate the magnitude of an action potential.  This is intedended to be an intermediate step with the end goal being to establish dose response curves plotting the magnitude of response against the amplitude of stimulation.

In [ ]:
import pyeCAP
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
sns.set_context("paper")

#### Set the file_location to the included sample data and import data using the Ephys class.
Note: You'll see some additional variables here that account for the delay between the onset of the recording and the onset of the stimulus.

In [ ]:
file_location = tdt_tanks = r"C:\Users\malal\Documents\WITNE\pyCAP\TutorialExamples\pnpig191126-191204-151801"
data_ephys = pyeCAP.Ephys(data=file_location, stores=['RawE', 'RawG'])

#### Groom  & Filter the Data

In [ ]:
# Groom Data
data_ephys = data_ephys.remove_ch('RawG 4')
rec_ch_names = ['LIFE 1', 'LIFE 2', 'LIFE 3', 'LIFE 4', 'EMG 1', 'EMG 2', 'EMG 3']
rec_ch_types = ['LIFE', 'LIFE', 'LIFE', 'LIFE', 'EMG', 'EMG', 'EMG']
data_ephys = data_ephys.set_ch_names(rec_ch_names)
data_ephys = data_ephys.set_ch_types(rec_ch_types)

# Filter Data
data_ephys = data_ephys.filter_powerline()
data_ephys = data_ephys.filter_median(btype='highpass')
data_ephys = data_ephys.filter_gaussian(Wn=4000, btype='lowpass')

#### Initiate an instance of the Stim class (this will get passed to the ECAP class, which does our calculations)

In [ ]:
data_stim = pyeCAP.Stim(file_location)

#### Initiate an instance of the ECAP class (this takes as an argument the distances from the stimulating electrode to the recording electrode) and average the data across stimulation trains, per stimulation

In [ ]:
data_ecap = pyeCAP.ECAP(data_ephys, data_stim, distance_log=[7.7, 8.3, 8.5, 8.6])
data_ecap.average_data()

#### The method calculate_AUC within the ECAP class will calculate the area under the curve for the entire averaged data

In [ ]:
data_ecap.calculate_AUC()

#### Let's pull this data into a pandas DataFrame and take a look

In [ ]:
df = data_ecap.master_df
df.head()

##### You'll note that our stimulation amplitudes (pulse amplitude (μA))  are all negative. This is the convention for cathodic based stimulation.  Let's change these from negative to positive to make figures more intuitive

In [ ]:
df['pulse amplitude (μA)'] = df['pulse amplitude (μA)'] * -1
df.head()

#### Now, let's make a nice plot of our A-alphas

In [ ]:
fig, ax = plt.subplots(1, dpi=300, figsize=(4,9./4))

sns.lineplot(x="pulse amplitude (μA)", y='AUC (Vs)',
             marker='o',
             estimator=np.mean,
             ci=0,
             hue="channel",
             data=df[(df["Calculation Window"] == "A-alpha") &
                     (df["Calculation Type"] == "RMS")], ax=ax)

ax.set_title('A-α Dose Response')
plt.tight_layout()
plt.show()

In [ ]:
data_stim = pyeCAP.Stim(file_location)

In [ ]:
data_ecap=pyeCAP.ECAP(data_ephys, data_stim)

In [ ]:
data_ecap.plot_channel("Stim 1", [(0,1)])

In [ ]:
data_stim.parameters